# Twitter API v1.1 Client

In [1]:
import tweepy
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

## Autentication

In [2]:
def get_auth_twitter():
        
    consumer_key= 'drhztkGNhICDt3cwBGXi7zzQk'
    consumer_secret= 'FDeCBlSi4qIbR9oPsf3tphQjCO6HqZfYANWceYIdJDZHsOaabi'
    access_token= '207778119-E9W1kqRXBbNeU2n8vcA0OoMn9XZpE9yvmZrD8gUx'
    access_token_secret= 'dncbYQQrh5jlJ97OmgxeaGgqOFiC7jnNpXdPngWMCRtDE'
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

    return api


## Query API V1

In [3]:
def search_for_tweets(api, search_words, until, lang, items):

        ##search_words = "DisneyPlus"
        ##until= "2020-11-03"
        ##lang = 'pt'

        cursor = tweepy.Cursor(api.search,
                    q=search_words,
                    lang=lang,
                    until= until, count=100, tweet_mode='extended').items(items)

        return cursor

## Cursor to List

In [4]:
def iterator_to_list(cursor):

    return list(cursor)      

In [5]:
pd.set_option('display.max_colwidth', None)

## Creating a DF

In [6]:

def tweets_to_dataframe(tweets_list):

    columns = set()
    
    tweets_data = []

    
    for tweet in tweets_list:
        keys = tweet._json.keys()
        single_data = {}
        
        
        for k in keys:

            valor = tweet._json.get(k)

            if tweet._json.get(k) == None:

                try:

                    single_data[k] = 0

                except Exception as e:
                    print(e)
                
            
            else:
              
               try:
                    single_data[k] = tweet._json.get(k)
               
               except Exception as e:
                    print(e)
                    single_data[k] = 0

                
            
            columns.add(k)

        tweets_data.append(single_data)    
    

    header_cols = list(columns)
    df = pd.DataFrame(tweets_data, columns=header_cols)
    df.head()
    
    return df

## Extracting data from objects

Entities column: finding mentions

In [7]:
def mentions(df):

        df['screen_name_mention_1'] = df['entities'].apply(lambda x: x['user_mentions'][0]['screen_name'] if x['user_mentions'] else None)

        df['id_mention_1'] = df['entities'].apply(lambda x: x['user_mentions'][0]['id_str'] if x['user_mentions'] else 0)

        return df

Retweeted_Status - Finding the author of the original tweet

In [8]:
def retweets(df):

    df['retweeted_from_screen_name'] = df['retweeted_status'].apply(lambda x: x['user']['screen_name'] if x is not np.nan else None)

    df['retweeted_from_id'] = df['retweeted_status'].apply(lambda x: x['user']['id_str'] if x is not np.nan else 0)

    df['retweeted_from_is_verified'] = df['retweeted_status'].apply(lambda x: x['user']['verified'] if x is not np.nan else None)

    df['retweet_full_text'] = df['retweeted_status'].apply(lambda x: x['full_text'] if x is not np.nan else None)

    return df 

User - Information about the user

In [9]:
def user(df):

    df['user_screen_name'] = df['user'].apply(lambda x: x['screen_name'] if x is not np.nan else None)

    df['user_id'] = df['user'].apply(lambda x: x['id_str'] if x is not np.nan else 0)

    df['user_followers'] = df['user'].apply(lambda x: x['followers_count'] if x is not np.nan else None)

    df['user_following'] = df['user'].apply(lambda x: x['friends_count'] if x is not np.nan else None)

    df['user_is_verified'] = df['user'].apply(lambda x: x['verified'] if x is not np.nan else None)

    return df

In [10]:
## New Columns

def destrinchando_colunas(df):
    
    mentions(df)
    retweets(df)
    user(df)

    return df


## Formating Datetime

In [11]:
def format_created_at(df):

    df['created_at'] = df['created_at'].apply(lambda x: datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'))

    return df

## Creating files

In [12]:
def create_csv(df, file_name):
    
    df.to_csv(f'C:/Users/amand/Documents/Jupyter Notebooks/DisneyPlus/data_fulltext_csv/{file_name}.csv')

In [13]:
def create_pickle(df, file_name):

    df.to_pickle(f'C:/Users/amand/Documents/Jupyter Notebooks/DisneyPlus/data_fulltext_pkl/{file_name}.pkl')

## Columns Filter

In [14]:
def filter_columns(df):

    important_columns = ['user_screen_name', 'user_id', 'user_is_verified', 'retweeted_from_screen_name', 'retweeted_from_id', 'retweeted_from_is_verified', 'screen_name_mention_1' , 'id_mention_1', 'in_reply_to_screen_name', 'in_reply_to_user_id', 'created_at', 'id', 'full_text','retweet_full_text','retweet_count', 'favorite_count', 'user_followers','user_following']

    df = df[important_columns]

    return df


In [15]:
def filter_columns_graph_ids(df):

    graph_columns_ids = ['user_id', 'retweeted_from_id' , 'id_mention_1', 'in_reply_to_user_id',  'id']

    df = df[graph_columns_ids]

    return df

## Todo o processo

In [16]:
def pega_tweets(search_words, until, lang, items, file_name):

    api = get_auth_twitter()

    cursor = search_for_tweets(api, search_words, until, lang, items)

    tweets_list = iterator_to_list(cursor)

    df = tweets_to_dataframe(tweets_list)
    df = destrinchando_colunas(df)
    df = format_created_at(df)

    create_csv(df, file_name)
    create_pickle(df, file_name)
   
    return df



In [17]:
def pega_tweets_filtrado(search_words, until, lang, items, file_name):

    
    api = get_auth_twitter()
    
    cursor = search_for_tweets(api, search_words, until, lang, items)

    tweets_list = iterator_to_list(cursor)

    df = tweets_to_dataframe(tweets_list)
    df = destrinchando_colunas(df)
    df = format_created_at(df)
    df = filter_columns (df)

    create_csv(df, file_name)
    create_pickle(df, file_name)
   
    return df


## Project Name: DisneyPlus

In [18]:
df = pega_tweets_filtrado("DisneyPlus", "", "pt", 80000, 'until_19_11')

Rate limit reached. Sleeping for: 513


In [19]:
df.created_at.sort_values()

62255   2020-11-10 10:12:09
62254   2020-11-10 10:14:04
62253   2020-11-10 10:55:23
62252   2020-11-10 10:55:40
62251   2020-11-10 10:55:43
                ...        
4       2020-11-19 13:57:25
3       2020-11-19 13:57:33
2       2020-11-19 13:57:56
1       2020-11-19 13:58:11
0       2020-11-19 13:58:15
Name: created_at, Length: 62256, dtype: datetime64[ns]

In [ ]:
break

In [64]:
api = get_auth_twitter()

In [65]:
cursor = search_for_tweets(api,"DisneyPlus", "", "pt", 10)

In [66]:
lista = list(cursor)

In [74]:
lista[0]._json.get('retweeted_status')

{'created_at': 'Tue Nov 17 12:31:47 +0000 2020',
 'id': 1328677211365404673,
 'id_str': '1328677211365404673',
 'full_text': 'O grande dia finalmente chegou. E este post talvez seja a última coisa que você vai ver antes de entrar no #DisneyPlus, então, aproveitando: use #QueroVerPrimeiro e conte pra gente o que você vai ver! 😉 https://t.co/2ctqjHlyNS',
 'truncated': False,
 'display_text_range': [0, 202],
 'entities': {'hashtags': [{'text': 'DisneyPlus', 'indices': [107, 118]},
   {'text': 'QueroVerPrimeiro', 'indices': [145, 162]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1328677174350635008,
    'id_str': '1328677174350635008',
    'indices': [203, 226],
    'media_url': 'http://pbs.twimg.com/media/EnBo4PlWEAAL1J3.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/EnBo4PlWEAAL1J3.jpg',
    'url': 'https://t.co/2ctqjHlyNS',
    'display_url': 'pic.twitter.com/2ctqjHlyNS',
    'expanded_url': 'https://twitter.com/DisneyPlusBR/status/1328677211365

In [71]:
cursor